In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re
from importlib import reload

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

from _lib.preprocess import preprocess_missing as prep
from _lib.preprocess import get_instruction as info

In [2]:
df = pd.read_csv("_database/Input/train.csv", index_col = 0)

df = prep(df)

C:\Anaconda\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [3]:
X = df.drop("SalePrice", axis = 1)
y = df["SalePrice"]

### Pipelines

In [18]:
df_type = pd.DataFrame({"Columns" : X.columns, "Type" : [str(X[col].dtype) for col in X.columns]})
num_columns = df_type.loc[(df_type["Type"] == "int64") | (df_type["Type"] == "float64")]["Columns"]
cat_columns = df_type.loc[(df_type["Type"] != "int64") & (df_type["Type"] != "float64")]["Columns"]

In [21]:
pd.DataFrame({"Num_col" : num_columns}).to_csv("numcol.csv")

In [5]:
list_categories = [np.array(info(col)) for col in cat_columns.values]
list_categories[14] = np.arange(1, 11)
list_categories[15] = np.arange(1, 11)

In [17]:
poly_degree = 1

In [13]:
get_numerical = FunctionTransformer(lambda x : x[num_columns.values].values,
                                    validate = False)
get_category = FunctionTransformer(lambda x : x[cat_columns.values], validate = False)
generate_poly = FunctionTransformer(lambda x : gen_poly(x, poly_degree), validate = False)

In [14]:
pipeline_num_prep = Pipeline([('selector', get_numerical),
                              ('poly', generate_poly)])

pipeline_cat_prep = Pipeline([('selector', get_category),
                              ('Dummy', OneHotEncoder(drop = 'first', sparse = False,
                                                     categories = list_categories))])

In [15]:
transformers = [ ('Numerical', pipeline_num_prep), ('Categorical', pipeline_cat_prep) ]

In [16]:
preprocess_union = FeatureUnion(transformer_list = transformers)

In [ ]:
pl = Pipeline([
    ('union', preprocess_union),
    ('linreg', LinearRegression())
])